# 00. Environment Validation

This notebook validates the Python environment and the hardware/software configuration for the federated learning lab. It checks the Python version, main packages, and hardware acceleration support (CPU, GPU, NPU).

---

## 1. Python Version and Interpreter Path

Check the Python version and the interpreter path used by the environment.

In [1]:
import sys


def print_python_info():
    """
    Prints the current Python version and interpreter path.
    """
    print(f"Python version: {sys.version}")
    print(f"Interpreter path: {sys.executable}")


print_python_info()


Python version: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
Interpreter path: C:\Users\padul\anaconda3\envs\ia-flower\python.exe


## 2. Main Package Versions

List and display the versions of the key packages for the lab.

In [14]:
import importlib


def print_package_versions(packages):
    """
    Prints the version of each package in the provided list. If not installed, prints an error message.
    Args:
        packages (list): List of package names as strings.
    """
    for pkg in packages:
        try:
            mod = importlib.import_module(pkg)
            ver = getattr(mod, '__version__', 'unknown')
            print(f"{pkg}: {ver}")
        except Exception as e:
            print(f"{pkg}: NOT INSTALLED ({e})")


main_packages = [
    'numpy', 'pandas', 'matplotlib', 'psutil', 'tqdm',
    'torch', 'flwr', 'openvino', 'openvino_genai', 'notebook',
    # Project-specific modules and their dependencies
    'sklearn', 'seaborn', 'tabulate',
]

print_package_versions(main_packages)

numpy: 2.3.2
pandas: 2.3.1
matplotlib: 3.10.3
psutil: 7.0.0
tqdm: 4.67.1
torch: 2.5.1
flwr: 1.19.0
openvino: 2025.2.0-19140-c01cd93e24d-releases/2025/2
openvino_genai: 2025.2.0.0-2260-01f0fe1eded
notebook: 7.4.4
sklearn: 1.7.1
seaborn: 0.13.2
tabulate: 0.9.0


## 3. Validate NVIDIA GPU and CUDA Version with PyTorch

Check if PyTorch detects the NVIDIA GPU and display the CUDA version.

In [3]:
import torch

gpu_available = torch.cuda.is_available()
print(f"Is GPU available (torch.cuda.is_available)? {gpu_available}")
if gpu_available:
    print(f"CUDA version (torch.version.cuda): {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("PyTorch does not detect an NVIDIA GPU.")

Is GPU available (torch.cuda.is_available)? True
CUDA version (torch.version.cuda): 12.6
Number of GPUs: 1
GPU name: NVIDIA GeForce RTX 4080 Laptop GPU


## 4. Device Detection with OpenVINO

List the available devices (CPU, GPU, NPU) using OpenVINO.

In [4]:
try:
    from openvino import Core

    core = Core()
    devices = core.available_devices
    print("Available devices in OpenVINO:")
    for d in devices:
        print(f" - {d}")
    print("\nDevice summary:")
    print(f"CPU detected: {any('CPU' in d for d in devices)}")
    print(f"GPU detected: {any('GPU' in d for d in devices)}")
    print(f"NPU (Intel AI Boost) detected: {any('NPU' in d or 'AUTO:NPU' in d for d in devices)}")
except Exception as e:
    print(f"Failed to detect OpenVINO devices: {e}")

Available devices in OpenVINO:
 - CPU
 - GPU
 - NPU

Device summary:
CPU detected: True
GPU detected: True
NPU (Intel AI Boost) detected: True


## 5. Validate scikit-learn

Validate the installation and configuration of scikit-learn, including version, main modules, and detailed configuration.

In [5]:
print("\n--- Validating scikit-learn ---")
try:
    import sklearn

    print(f"✔ scikit-learn imported successfully. Version: {sklearn.__version__}")
    # Check some main modules
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    print("✔ Main scikit-learn modules are available")
    # Show detailed configuration
    sklearn.show_versions()
except Exception as e:
    print(f"✗ Error validating scikit-learn: {e}")


--- Validating scikit-learn ---
✔ scikit-learn imported successfully. Version: 1.7.1
✔ Main scikit-learn modules are available

System:
    python: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
executable: C:\Users\padul\anaconda3\envs\ia-flower\python.exe
   machine: Windows-11-10.0.26100-SP0

Python dependencies:
      sklearn: 1.7.1
          pip: 25.1.1
   setuptools: 75.8.2
        numpy: 2.3.2
        scipy: 1.16.0
       Cython: None
       pandas: 2.3.1
   matplotlib: 3.10.3
       joblib: 1.5.1
threadpoolctl: 3.6.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: mkl
    num_threads: 1
         prefix: libblas
       filepath: C:\Users\padul\anaconda3\envs\ia-flower\Library\bin\libblas.dll
        version: 2025.2-Product
threading_layer: intel

       user_api: openmp
   internal_api: openmp
    num_threads: 1
         prefix: libiomp
       filepath: C:\Users\padul\anaconda3\envs\ia-flower\Lib

## 6. Environment Validation Summary

This cell summarizes the validation. If any critical component is missing, check the previous output and install or configure as necessary.

In [6]:
# Simple environment summary
missing = []

# Python version check
if sys.version_info < (3, 8):
    missing.append('Python >= 3.8 is required')

# Package availability check
for pkg in main_packages:
    try:
        importlib.import_module(pkg)
    except:
        missing.append(f"{pkg} is not installed")

# GPU availability with PyTorch
if not torch.cuda.is_available():
    missing.append('PyTorch does not detect NVIDIA GPU')

# OpenVINO device detection
try:
    ov_devices = core.available_devices
    if not any('CPU' in d for d in ov_devices):
        missing.append('OpenVINO does not detect CPU')
    if not any('GPU' in d for d in ov_devices):
        missing.append('OpenVINO does not detect GPU')
    if not any('NPU' in d or 'AUTO:NPU' in d for d in ov_devices):
        missing.append('OpenVINO does not detect NPU (Intel AI Boost)')
except:
    missing.append('Failed to detect OpenVINO devices')

# scikit-learn validation
try:
    import sklearn
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    sklearn.show_versions()
except Exception:
    missing.append('scikit-learn is not correctly installed or configured')

if missing:
    print('The environment is NOT ready. Detected issues:')
    for m in missing:
        print(f'- {m}')
else:
    print('The environment is READY for federated learning experiments!')



System:
    python: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
executable: C:\Users\padul\anaconda3\envs\ia-flower\python.exe
   machine: Windows-11-10.0.26100-SP0

Python dependencies:
      sklearn: 1.7.1
          pip: 25.1.1
   setuptools: 75.8.2
        numpy: 2.3.2
        scipy: 1.16.0
       Cython: None
       pandas: 2.3.1
   matplotlib: 3.10.3
       joblib: 1.5.1
threadpoolctl: 3.6.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: mkl
    num_threads: 1
         prefix: libblas
       filepath: C:\Users\padul\anaconda3\envs\ia-flower\Library\bin\libblas.dll
        version: 2025.2-Product
threading_layer: intel

       user_api: openmp
   internal_api: openmp
    num_threads: 1
         prefix: libiomp
       filepath: C:\Users\padul\anaconda3\envs\ia-flower\Library\bin\libiomp5md.dll
        version: None

       user_api: openmp
   internal_api: openmp
    num_threads: 22
         pref